# 🚀 AI Agents Bootcamp - Day 2
## Multi-Agent Interview Prep System with CrewAI

**What you'll build:** A 3-agent system that researches companies, analyzes your fit, and generates targeted interview prep materials.

**Time:** ~15 minutes setup, 2-3 minutes to run

**Prerequisites:** OpenAI API key (required), Serper API key (optional, for web search)

---

📚 **Full Series:** [Standout Systems on Substack](https://teodoracoach.substack.com/)  
💻 **GitHub:** [ai-agents-bootcamp](https://github.com/DoraSzasz/ai-agents-bootcamp)  
🎯 **Coaching:** [teodora.coach](https://teodora.coach)

## Step 1: Install Dependencies

In [ ]:
# Install required packages (takes ~30 seconds)
!pip install -q crewai crewai-tools langchain-openai python-dotenv

## Step 2: Set Up API Keys

**Option A (Recommended):** Use Colab Secrets
1. Click the 🔑 key icon in the left sidebar
2. Add `OPENAI_API_KEY` and optionally `SERPER_API_KEY`

**Option B:** Paste directly (less secure)

In [ ]:
import os

# Try to get keys from Colab secrets first
try:
    from google.colab import userdata
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
    
    # Serper is optional - for web search
    try:
        os.environ["SERPER_API_KEY"] = userdata.get('SERPER_API_KEY')
        print("✅ Both API keys loaded from Colab Secrets")
    except:
        print("✅ OpenAI key loaded. Serper not found (web search disabled)")
        
except:
    # Fallback: paste your keys directly
    os.environ["OPENAI_API_KEY"] = "sk-your-key-here"  # ← Replace this!
    os.environ["SERPER_API_KEY"] = "your-serper-key"   # ← Optional
    print("⚠️  Using hardcoded keys - add to Colab Secrets for security")

## Step 3: Import Libraries

In [ ]:
from crewai import Agent, Crew, Task, Process
from crewai_tools import SerperDevTool
from langchain_openai import ChatOpenAI
from datetime import datetime

print("✅ All libraries imported successfully!")

## Step 4: Configure Your Interview

**🎯 CUSTOMIZE THIS SECTION for your actual interview!**

In [ ]:
# ═══════════════════════════════════════════════════════════════
# 🎯 CUSTOMIZE THESE FOR YOUR INTERVIEW
# ═══════════════════════════════════════════════════════════════

COMPANY = "Anthropic"  # Your target company
POSITION = "Senior Machine Learning Engineer"  # The role
INTERVIEWER = "Chris Olah"  # From your calendar invite (if known)

JOB_DESCRIPTION = """
We're looking for ML engineers to work on interpretability research.
You'll develop techniques to understand neural network behavior,
work on scaling alignment research, and collaborate with researchers
on safety-focused ML systems. Requirements: 5+ years ML experience,
strong Python, experience with transformer architectures.
"""

YOUR_BACKGROUND = """
- 6 years ML engineering at tech companies
- Built production transformer models for NLP
- Published paper on attention visualization
- Led team of 4 ML engineers
- Strong Python, PyTorch, distributed training
"""

print(f"🎯 Preparing for: {POSITION} at {COMPANY}")

## Step 5: Initialize LLM and Tools

The 5 Building Blocks in multi-agent systems:

| Block | Single Agent (Day 1) | Multi-Agent (Day 2) |
|-------|---------------------|---------------------|
| 🎯 Goal | One goal | Specialized goals per agent |
| 🧠 Reasoning | One LLM | Same LLM, different prompts |
| 🔧 Tools | None | Web search + more |
| 💾 Memory | Simple dict | Shared via task context |
| 🔄 Feedback | Validation | Cross-agent review |

In [ ]:
# Initialize the LLM (shared by all agents)
llm = ChatOpenAI(model="gpt-4o", temperature=0.7)

# Initialize tools (web search if available)
tools = []
if os.getenv("SERPER_API_KEY"):
    tools.append(SerperDevTool())
    print("✅ Web search enabled - agents can search the internet")
else:
    print("⚠️  No Serper key - running without web search")
    print("   Get free key at https://serper.dev (2500 free searches)")

## Step 6: Create Your Agent Team

Each agent is a specialist with unique expertise:

```
┌─────────────────┐     ┌─────────────────┐     ┌─────────────────┐
│ 🔍 RESEARCHER   │────▶│ 🎯 ANALYZER     │────▶│ 💬 COACH        │
│ Company Intel   │     │ Fit & Strategy  │     │ Questions       │
└─────────────────┘     └─────────────────┘     └─────────────────┘
```

In [ ]:
# ═══════════════════════════════════════════════════════════════
# AGENT 1: THE RESEARCHER
# Building Block: Tools + Deep Research Goal
# ═══════════════════════════════════════════════════════════════
research_agent = Agent(
    role="Senior Company Research Analyst",
    goal=f"Conduct comprehensive research on {COMPANY} "
         f"to prepare candidate for interview success",
    backstory="""You are an elite research analyst who has helped hundreds 
    of candidates land jobs at top companies. You know exactly what information 
    matters: recent news, company culture, technical challenges, and the 
    specific team dynamics. You dig deep and never settle for surface-level 
    information. You cite your sources and focus on actionable insights.""",
    llm=llm,
    tools=tools,
    verbose=True,
    max_iter=5
)

# ═══════════════════════════════════════════════════════════════
# AGENT 2: THE FIT ANALYZER
# Building Block: Strategic Reasoning
# ═══════════════════════════════════════════════════════════════
analyzer_agent = Agent(
    role="Career Strategy Consultant",
    goal="Analyze candidate-company fit and develop winning positioning strategy",
    backstory="""You are a career strategist who has coached executives at 
    FAANG companies. You excel at finding the perfect angle to present 
    someone's experience. You see connections others miss—how a candidate's 
    past projects align with a company's current challenges. You think in 
    terms of compelling stories, not bullet points.""",
    llm=llm,
    verbose=True
)

# ═══════════════════════════════════════════════════════════════
# AGENT 3: THE INTERVIEW COACH
# Building Block: Synthesized Output
# ═══════════════════════════════════════════════════════════════
coach_agent = Agent(
    role="Technical Interview Coach",
    goal=f"Generate highly targeted interview questions and prep strategy for "
         f"{POSITION} role",
    backstory="""You've conducted 500+ technical interviews at top tech 
    companies. You know exactly what interviewers look for at each level.
    You create questions that are specific, realistic, and actually 
    prepare people for success—not generic fluff. You think about the 
    interviewer's perspective and what they're really trying to assess.""",
    llm=llm,
    verbose=True
)

print("✅ Created 3 specialized agents: Researcher, Analyzer, Coach")

## Step 7: Define the Task Pipeline

Tasks chain together - each task can access previous task outputs via `context`.

In [ ]:
# ═══════════════════════════════════════════════════════════════
# TASK 1: DEEP COMPANY RESEARCH
# ═══════════════════════════════════════════════════════════════
research_task = Task(
    description=f"""
    Research {COMPANY} comprehensively for an interview.
    
    Cover these areas IN DEPTH:
    
    1. **Recent News** (last 6 months)
       - Major announcements, funding, product launches
       - Challenges or controversies
       - Key wins and milestones
    
    2. **Technical Focus**
       - What problems are they solving?
       - Tech stack and approach
       - Recent engineering work or papers
    
    3. **Team & Culture**
       - Leadership backgrounds
       - Real culture (not just PR)
       - What employees actually say
    
    4. **The Role: {POSITION}**
       - What team is this likely on?
       - Current projects and challenges
       - What success looks like
    
    5. **Interviewer: {INTERVIEWER}**
       - Their background and expertise
       - Published work or public talks
       - What they care about
    
    Be specific. Cite sources. Skip the generic.
    """,
    expected_output="Detailed research report with executive summary, findings by category, and sources",
    agent=research_agent
)

# ═══════════════════════════════════════════════════════════════
# TASK 2: FIT ANALYSIS & POSITIONING
# ═══════════════════════════════════════════════════════════════
analysis_task = Task(
    description=f"""
    Based on the research, analyze candidate fit and create positioning strategy.
    
    **Candidate Background:**
    {YOUR_BACKGROUND}
    
    **Job Description:**
    {JOB_DESCRIPTION}
    
    Deliver:
    1. **Strongest Alignment Points** - Experience that matches needs
    2. **Gap Analysis** - What interviewer might worry about + mitigation
    3. **Unique Value Proposition** - The memorable angle
    4. **Prepared Stories** - 3-5 STAR-format examples to have ready
    5. **Things to Avoid** - Topics to steer away from
    """,
    expected_output="Strategic positioning brief with alignment mapping and prepared stories",
    agent=analyzer_agent,
    context=[research_task]  # ← Uses research output!
)

# ═══════════════════════════════════════════════════════════════
# TASK 3: INTERVIEW PREP QUESTIONS
# ═══════════════════════════════════════════════════════════════
coaching_task = Task(
    description=f"""
    Create comprehensive interview prep for {POSITION} at {COMPANY}.
    
    Generate:
    
    1. **Technical Questions** (10) - Specific to their tech and problems
    2. **Behavioral Questions** (8) - Aligned with company values
    3. **Role-Specific Scenarios** (5) - "How would you approach X?"
    4. **Questions to ASK Them** (5) - Smart, researched questions
    5. **Interview Day Strategy** - Opening script, closing, energy tips
    
    Every question must be SPECIFIC to this company. Zero generic questions.
    Include answer frameworks for each question.
    """,
    expected_output="Complete interview prep guide with categorized questions and strategies",
    agent=coach_agent,
    context=[research_task, analysis_task]  # ← Uses both previous outputs!
)

print("✅ Created 3 chained tasks: Research → Analysis → Coaching")

## Step 8: Assemble and Run the Crew 🚀

In [ ]:
# Assemble the crew
interview_crew = Crew(
    agents=[research_agent, analyzer_agent, coach_agent],
    tasks=[research_task, analysis_task, coaching_task],
    process=Process.sequential,  # Tasks run in order
    verbose=True
)

# Launch!
print("=" * 60)
print("🚀 INTERVIEW PREP SYSTEM ACTIVATED")
print("=" * 60)
print(f"📍 Company:     {COMPANY}")
print(f"💼 Position:    {POSITION}")
print(f"👤 Interviewer: {INTERVIEWER}")
print("=" * 60)
print("\n⏳ Running 3-agent pipeline (2-4 minutes)...")
print("   Watch the agents think below!\n")

result = interview_crew.kickoff()

## Step 9: View Your Results

In [ ]:
from IPython.display import Markdown, display

print("\n" + "=" * 60)
print("📋 YOUR INTERVIEW PREP GUIDE")
print("=" * 60 + "\n")

# Display as formatted markdown
display(Markdown(result.raw))

## Step 10: Save Your Prep Guide

In [ ]:
# Save to file
timestamp = datetime.now().strftime("%Y%m%d_%H%M")
filename = f"{COMPANY.lower().replace(' ', '_')}_interview_prep_{timestamp}.md"

report = f"""# Interview Prep Guide: {COMPANY}
## {POSITION}

**Generated:** {datetime.now().strftime("%Y-%m-%d %H:%M")}
**Interviewer:** {INTERVIEWER}

---

{result.raw}

---

*Generated by AI Agents Bootcamp - Day 2*
*https://github.com/DoraSzasz/ai-agents-bootcamp*
"""

with open(filename, 'w') as f:
    f.write(report)

print(f"💾 Saved to: {filename}")

# Download link for Colab
try:
    from google.colab import files
    files.download(filename)
    print("📥 Download started!")
except:
    print("(Run in Colab to auto-download)")

---

## 🎯 Experiments to Try

**Easy:**
- Change `COMPANY`, `POSITION`, and `YOUR_BACKGROUND` to your actual interview
- Adjust `temperature` (0.5 = more focused, 0.9 = more creative)

**Medium:**
- Add a 4th agent: "Mock Interviewer" that asks follow-up questions
- Modify tasks to focus on specific interview rounds (phone screen vs. onsite)

**Advanced:**
- Add `ScrapeWebsiteTool` for deeper research
- Build a Streamlit UI around this
- Chain into a job application tracker system

---

## 📚 Day 3 Preview: LangGraph

Tomorrow we build **interactive loops** - an agent that actually drills you on interview questions and adapts based on your answers.

That's when agents get seriously powerful.

**Subscribe:** [Standout Systems](https://teodoracoach.substack.com/)  
**GitHub:** [ai-agents-bootcamp](https://github.com/DoraSzasz/ai-agents-bootcamp)  
**Coaching:** [teodora.coach](https://teodora.coach)